<h1 align="center">
    <b style="color: green;">Trino </b><b style="color: red;">Iceberg </b><b style="color: black;">Example</b>
</h1>

---

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.schema import Table, MetaData
from sqlalchemy.sql.expression import select, text
import pandas as pd

In [2]:
engine = create_engine('trino://root:@192.168.10.210:8090/iceberg/iceberg')
connection = engine.connect()

## Re-Snapshot

In [4]:
connection.execute("CALL iceberg.system.rollback_to_snapshot('iceberg_test', 'tbl_example', 822704853044330400)")

## Query เพื่อดูประวัติการ Snapshot

In [5]:
sql = '''
    SELECT
        *
    FROM
        iceberg.iceberg_test."tbl_example$snapshots"
    ORDER BY
        committed_at DESC
'''
pd.read_sql_query(sql, con=engine)

,committed_at,snapshot_id,parent_id,operation,manifest_list,summary
0,2023-03-19 05:50:15.954000+00:00,822704853044330400,1.554897e+18,append,hdfs://nn01.bigdata:8020/user/hive/warehouse/i...,"{'changed-partition-count': '1', 'added-data-f..."
1,2023-03-18 07:49:13.730000+00:00,1554896567181628286,7.808855e+18,append,hdfs://nn01.bigdata:8020/user/hive/warehouse/i...,"{'changed-partition-count': '1', 'added-data-f..."
2,2023-03-18 07:49:02.816000+00:00,7808855038413598624,8.700949e+18,append,hdfs://nn01.bigdata:8020/user/hive/warehouse/i...,"{'changed-partition-count': '1', 'added-data-f..."
3,2023-03-18 07:47:36.388000+00:00,8700948751126317934,NaN,append,hdfs://nn01.bigdata:8020/user/hive/warehouse/i...,"{'changed-partition-count': '0', 'total-equali..."


## แสดงรายการ Record หรือ Rows ทั้งหมดในตารางพร้อมระบุเวลาที่เกิด Condition พร้อมบอกว่า ข้อมูล Row นี้อยู่ในไฟล์ parquet ตัวไหน

In [12]:
sql = '''
    SELECT
        *, 
        "$path", "$file_modified_time"
    FROM
        iceberg.iceberg_test.tbl_example
'''
pd.read_sql_query(sql, con=engine)

,_c0,age_group,vaccine_status,outcome,$path,$file_modified_time
0,5,test5,test5,test5,hdfs://nn01.bigdata:8020/user/hive/warehouse/i...,2023-03-19 05:32:17.554000+00:00
1,2,test2,test2,test2,hdfs://nn01.bigdata:8020/user/hive/warehouse/i...,2023-03-18 07:31:14.581000+00:00
2,1,test,test,test,hdfs://nn01.bigdata:8020/user/hive/warehouse/i...,2023-03-18 07:31:03.638000+00:00


## Query แบบระบุไฟล์ Parquet

In [11]:
sql = '''
    SELECT
        *
    FROM
        iceberg.iceberg_test.tbl_example
    WHERE
        "$path" = 'hdfs://nn01.bigdata:8020/user/hive/warehouse/iceberg_test/tbl_example/data/20230318_073101_00095_hdwqq-ae0b490d-8fd9-46ab-aed5-526bf34f0ea9.parquet'
'''
pd.read_sql_query(sql, con=engine)

,_c0,age_group,vaccine_status,outcome
0,1,test,test,test
